In [11]:
from pyspark.sql import *
from collections import defaultdict
from itertools import combinations
import math

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/05/16 09:33:21 WARN Utils: Your hostname, MacBook-Pro-Sofi.local resolves to a loopback address: 127.0.0.1; using 109.123.185.23 instead (on interface en6)
23/05/16 09:33:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 09:33:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.csv('Datasets/FIFA.csv', sep=',', header=True, inferSchema=True, multiLine=True, escape='"')

In [4]:
rdd = data.rdd

In [5]:
top_users = (
    rdd.filter(lambda x: x.UserMentionID is not None)
    .flatMap(lambda x: x.UserMentionID.lower().split(',')) 
    .map(lambda x: (x, 1)) 
    .reduceByKey(lambda x, y: x + y) 
    .top(5, key=lambda x: x[1])  
)
print(top_users)



[('fifaworldcup', 91122), ('khaledbeydoun', 17507), ('weareoneexo', 14974), ('hns_cff', 11836), ('paulpogba', 11465)]


In [6]:
users_tags = []
for user, _ in top_users:
    tweets_from_user = rdd.filter(lambda x: x.UserMentionID is not None and user in x.UserMentionID.lower().split(','))
    tags_from_user = tweets_from_user.flatMap(lambda x: x.Hashtags.lower().split(',') if x.Hashtags is not None else [])
    tags_from_user = tags_from_user.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
    top_tags = tags_from_user.top(10, key=lambda x: x[1])
    users_tags.append((user, top_tags))
    print((user, top_tags))

print(type(users_tags))
print(users_tags)

('fifaworldcup', [('worldcup', 77249), ('fra', 24828), ('fifastadiumdj', 13563), ('worldcupfinal', 13551), ('fracro', 9147), ('cro', 8565), ('powerbyexo', 7325), ('bel', 6449), ('frabel', 5927), ('exo', 4679)])


('khaledbeydoun', [('worldcup', 17432), ('france', 76), ('belgium', 12), ('fra', 2), ('african', 1), ('sports', 1), ('arts', 1), ('music', 1), ('technology', 1), ('ideas', 1)])


('weareoneexo', [('worldcup', 11366), ('fifastadiumdj', 10516), ('powerbyexo', 7217), ('exo', 5037), ('??', 2910), ('dafbama2018exo', 1148), ('power', 585), ('copa2018', 547), ('chanyeolleague', 365), ('layleague', 358)])


('hns_cff', [('worldcup', 10094), ('cro', 8429), ('croeng', 2363), ('beproud', 1837), ('family', 1489), ('vatreni', 1472), ('croden', 1231), ('flamingpride', 886), ('croatia', 844), ('worldcupfinal', 788)])


('paulpogba', [('worldcup', 11327), ('fra', 2900), ('heretocreate', 2438), ('cro', 621), ('bel', 594), ('mufc', 187), ('fraarg', 129), ('worldcupfinal', 108), ('u18pl', 87), ('arg', 76)])
<class 'list'>
[('fifaworldcup', [('worldcup', 77249), ('fra', 24828), ('fifastadiumdj', 13563), ('worldcupfinal', 13551), ('fracro', 9147), ('cro', 8565), ('powerbyexo', 7325), ('bel', 6449), ('frabel', 5927), ('exo', 4679)]), ('khaledbeydoun', [('worldcup', 17432), ('france', 76), ('belgium', 12), ('fra', 2), ('african', 1), ('sports', 1), ('arts', 1), ('music', 1), ('technology', 1), ('ideas', 1)]), ('weareoneexo', [('worldcup', 11366), ('fifastadiumdj', 10516), ('powerbyexo', 7217), ('exo', 5037), ('??', 2910), ('dafbama2018exo', 1148), ('power', 585), ('copa2018', 547), ('chanyeolleague', 365), ('layleague', 358)]), ('hns_cff', [('worldcup', 10094), ('cro', 8429), ('croeng', 2363), ('beproud', 1837), ('family', 1489), ('vatreni', 1472), ('croden', 1231), ('flamingpride', 886), ('croatia', 844), (

In [7]:
tag_sets = [set(map(lambda x: x[0], tags)) for _, tags in users_tags]
most_different_user = None
most_different_tags = None
most_different_score = 0

for i, tag_set in enumerate(tag_sets):
    score = sum(len(tag_set.symmetric_difference(other_tag_set)) for j, other_tag_set in enumerate(tag_sets) if i != j)
    if score > most_different_score:
        most_different_user = top_users[i][0]
        most_different_tags = tag_set
        most_different_score = score

print(most_different_user, most_different_tags)


khaledbeydoun {'belgium', 'ideas', 'music', 'technology', 'worldcup', 'france', 'arts', 'sports', 'african', 'fra'}


In [12]:
users = set()
tags = set()

for user, user_tags in users_tags:
    users.update([user])
    user_tags = set(map(lambda x: x[0], user_tags))
    tags.update(user_tags)

users = list(users)
tags = list(tags)

dict_users_tags = {user: defaultdict(int) for user in users}

def update_dict_users_tags(row, dict_users_tags, users, tags):
    if row.UserMentionID is None or row.Hashtags is None:
        return
    for user in row.UserMentionID.lower().split(','):
        if user not in users:
            continue
        for tag in row.Hashtags.lower().split(','):
            if tag not in tags:
                continue
            dict_users_tags[user][tag] += 1


for _, row in data.toPandas().iterrows():
    update_dict_users_tags(row, dict_users_tags, users, tags)

min_user_1 = None
min_user_2 = None
min_dist = math.inf

for user_1, user_2 in combinations(users, 2):
    dist = sum((dict_users_tags[user_1][tag] - dict_users_tags[user_2][tag]) ** 2 for tag in tags)
    dist = math.sqrt(dist)

    if dist < min_dist:
        min_dist = dist
        min_user_1 = user_1
        min_user_2 = user_2

print(min_user_1)
print(min_user_2)
print(min_dist)

khaledbeydoun
paulpogba
7235.007532822616
